<a href="https://colab.research.google.com/github/HarindraMavikumbure/Twitter_sentiment/blob/main/Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji
!pip install transformers
!pip install nltk emoji==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 6.8 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=11421d730e8fe4d15300faaff3b3fd513818754c806b777d987a27d35c3f37d7
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 8.3 MB/s 
     |████████████████████████████████| 163 kB 65.0 MB/s 
     |████████████████████████████████| 7.6 MB 52.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 219 kB/s 
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49734 sha256=945de401e00ee5b439b17cfd919d31a693fa281680ad699f57b51

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import re, string
import emoji
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split


#Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#keras
import tensorflow as tf
from tensorflow import keras


#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

from emoji import demojize

#set seed for reproducibility
seed=42

#set style for plots
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)


<Figure size 432x288 with 0 Axes>

In [ ]:
df_train = pd.read_csv('groups_balanced.csv')
df_test = pd.read_csv('groups_test.csv',encoding='ISO-8859-1')

In [ ]:
df_train.head()

,clean_text,category
0,@Mysonne @NYjusticeleague & Faith Leaders #Shu...,0
1,Is the #American #CriminalJustice system broke...,-1
2,@DebsConlon @coinabs @BobbyMc2014 @israel_shie...,0
3,Ebad Hussain @BrownMaaamba says #Hamas rockets...,-1
4,PURE COMEDY. From a supporter of #Hamas and Is...,0


In [ ]:
df_test.head()

,clean_text,category,text_clean,text_len,token_lens
0,#klassfeeronderdruk bcs Selena attackeerd mevr...,NaN,klassfeeronderdruk bcs selena attackeerd mevr ...,8,23
1,Where's the best place to buy a @utahjazz jers...,NaN,wheres the best place to buy a jersey,8,11
2,@DavidDJJames @utahjazz But the @Runnin_Utes g...,NaN,but the game starts at 7 dont make me choose d...,12,15
3,@Fazzino @Cubs A Christmas caro by Charles Di...,NaN,a christmas caro by charles dickens l sundaymo...,8,14
4,Obama's latest #BlackLivesMatter recruiting vi...,NaN,obamas latest blacklivesmatter recruiting vide...,9,20


In [ ]:
df = df_train[['clean_text','category']]
df_test = df_test[['clean_text','category']]

In [ ]:
##CUSTOM DEFINED FUNCTIONS TO CLEAN THE TWEETS

#Clean emojis from text
def strip_emoji(text):
    return demojize(text) #remove emoji

#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [ ]:
texts_new = []
for t in df.clean_text:
    texts_new.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))

In [ ]:
texts_new_test = []
for t in df_test.clean_text:
    texts_new_test.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))

In [ ]:
df['text_clean'] = texts_new
df_test['text_clean'] = texts_new_test

In [ ]:
text_len = []
for text in df.text_clean:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

In [ ]:
df['text_len'] = text_len

In [ ]:
text_len_test = []
for text in df_test.text_clean:
    tweet_len = len(text.split())
    text_len_test.append(tweet_len)

In [ ]:
df_test['text_len'] = text_len_test

In [ ]:
df['text_clean'].head()
df_test['text_clean'].head()

0    soros brainwashes enslaves u blacklivesmatter ...
1    obamas latest blacklivesmatter recruiting vide...
2    theres not enough hair to track all the death ...
3    the latest the humanitarian daily blacklivesma...
4    leftist smurf defaces landmark with anti trump...
Name: text_clean, dtype: object

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
token_lens = []

for txt in df['text_clean'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    
max_len=np.max(token_lens)

In [ ]:
print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 52


In [ ]:
token_lens = []

for i,txt in enumerate(df['text_clean'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}") 

In [ ]:
df['token_lens'] = token_lens

In [ ]:
df = df.sort_values(by='token_lens', ascending=False)
df.head(20)

,clean_text,category,text_clean,token_lens
972,@sportsdes that's amazing😎. I am a tipp suppor...,1,thats amazingsmilingfacewithsunglasses i am a ...,52
542,Liam...just...OMG!❤️😍 #ARIASONEDIRECTION #oned...,1,liamjustomgredheartsmilingfacewithhearteyes ar...,44
553,Happy Thanksgiving Everyone 😂😂😂 🐥#thanksgiving...,1,happy thanksgiving everyone facewithtearsofjoy...,43
890,The other day I went on a date & this guy aske...,0,the other day i went on a date this guy asked ...,42
755,@realDonaldTrump's #TrumpTransitionTeam Runnin...,0,trumptransitionteam runnin the country redexcl...,42
843,@naenae_calhoun @Tayla_Gipson lmaooo all day e...,1,lmaooo all day everyday doubleexclamationmark ...,41
227,Give it a listen! Raw project from one of the ...,0,give it a listen raw project from one of the r...,40
740,#FuckTrump @MicroRubio America's Nightmare. 🇺🇸...,-1,fucktrump americas nightmare unitedstatesfaces...,40
586,after I finish dinner I'm gonna jam to mm like...,1,after i finish dinner im gonna jam to mm like ...,40
116,"Has #Obama ever apologized for the terrorists,...",-1,has obama ever apologized for the terrorists a...,40


In [ ]:
df = df.iloc[12:]
df.head()

,clean_text,category,text_clean,token_lens
263,Trump should be made to cough up #reparations ...,1,trump should be made to cough up reparations k...,39
857,Cubs lineup 2017:LF SchwarbsCF AlmoraRF JHey3B...,0,cubs lineup 2017lf schwarbscf almorarf jhey3b ...,38
660,On my way to the @DGVN_de #youth conference #u...,2,on my way to the youth conference up2youth glo...,38
532,Retweet and tweet this 👇👇👇#MPN #OneDirection,0,retweet and tweet this backhandindexpointingdo...,38
48,America's Nightmare. 🇺🇸😱💤#trumptransitionteam ...,-1,americas nightmare unitedstatesfacescreamingin...,38


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
token_lens_test = []

for txt in df_test['text_clean'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens_test.append(len(tokens))
    
max_len=np.max(token_lens_test)

In [ ]:
token_lens_test = []

for i,txt in enumerate(df_test['text_clean'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens_test.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}")

In [ ]:
df_test['token_lens'] = token_lens_test

In [ ]:
df_test = df_test.sort_values(by='token_lens', ascending=False)
df_test.head(10) 

,clean_text,category,text_clean,text_len,token_lens
63,The swamp is bought AND paid for-Major #gop do...,0,the swamp is bought and paid formajor gop dono...,16,32
10,"Has #Obama ever apologized for the terrorists,...",-1,has obama ever apologized for the terrorists a...,21,31
89,FINAL- Fairmont 38 Tippecanoe 28. Allison Made...,0,final fairmont 38 tippecanoe 28 allison mader ...,16,31
32,Zayn's name in Arabic means 'beautiful' #fact ...,1,zayns name in arabic means beautiful fact oned...,13,30
5,smdh this country cares more about black frida...,0,smdh this country cares more about black frida...,22,30
96,Overt #KremlinTrolls will ReTweet covert ones ...,-1,overt kremlintrolls will retweet covert ones w...,18,30
40,Whatthefuck Liam OMG #liampayne #onedirection ...,-1,whatthefuck liam omg liampayne onedirection ho...,11,28
64,The @utahjazz were on fire tonight! It was awe...,1,the were on fire tonight it was awesome to be ...,26,28
71,"Anthony Rizzo Chicago Cubs Autographed 8"" x 10...",0,anthony rizzo chicago cubs autographed 8 x 10 ...,13,28
0,Soros brainwashes & enslaves U #blacklivesmatt...,-1,soros brainwashes enslaves u blacklivesmatter ...,16,28


In [ ]:
df_test = df_test.iloc[5:]
df_test.head(3)

,clean_text,category,text_clean,text_len,token_lens
5,$HE $YELP $MY $FIG $TWO $CUBS $AGO$IM $MS $AXE...,NaN,he yelp my fig two cubs agoim ms axe at a zeni...,14,19
6,WOW! LISTEN To The Hate: Black Girl's White Bo...,NaN,wow listen to the hate black girls white boyfr...,13,16
7,War & Cyber #SDF #YPG #ISIS #News https://t.c...,NaN,war cyber sdf ypg isis news,6,10


In [ ]:
df_test = df_test.sample(frac=1).reset_index(drop=True)

In [ ]:
df['category'].value_counts()

Series([], Name: category, dtype: int64)

In [ ]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['text_clean']).reshape(-1, 1), np.array(df['category']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['text_clean', 'category']);

ValueError: ignored

In [ ]:
X = train_os['text_clean'].values
y = train_os['category'].values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, stratify=y, random_state=seed)

In [ ]:
X_test = df_test['text_clean'].values
y_test = df_test['category'].values

In [ ]:
y_train_le = y_train.copy()
y_valid_le = y_valid.copy()
y_test_le = y_test.copy()

In [ ]:
X_test = df_test['text_clean'].values
y_test = df_test['category'].values

In [ ]:
ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1)).toarray()
y_test = ohe.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()

In [ ]:
clf = CountVectorizer()
X_train_cv =  clf.fit_transform(X_train)
X_test_cv = clf.transform(X_test)

In [ ]:
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_cv)
X_train_tf = tf_transformer.transform(X_train_cv)
X_test_tf = tf_transformer.transform(X_test_cv)

In [ ]:
nb_clf = MultinomialNB()

In [ ]:
nb_clf.fit(X_train_tf, y_train_le)

MultinomialNB()

In [ ]:
nb_pred = nb_clf.predict(X_test_tf)

In [ ]:
print('\tClassification Report for Naive Bayes:\n\n',classification_report(y_valid_le,nb_pred, target_names=['High negetive','Negative', 'Neutral', 'Positive', 'High positive']))

	Classification Report for Naive Bayes:

                precision    recall  f1-score   support

High negetive       0.91      1.00      0.95        60
     Negative       0.72      0.92      0.81        59
      Neutral       0.95      0.30      0.46        60
     Positive       0.80      0.98      0.88        60
High positive       0.92      1.00      0.96        59

     accuracy                           0.84       298
    macro avg       0.86      0.84      0.81       298
 weighted avg       0.86      0.84      0.81       298



In [ ]:
MAX_LEN=128

In [ ]:
def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(X_valid, MAX_LEN)
test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [ ]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_model(bert_model, max_len=MAX_LEN):
    
    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    embeddings = bert_model([input_ids,attention_masks])[1]
    
    output = tf.keras.layers.Dense(5, activation="softmax")(embeddings)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)
    
    model.compile(opt, loss=loss, metrics=accuracy)
    
    
    return model

In [ ]:
model = create_model(bert_model, MAX_LEN)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_3[0][0]',                
                                thPoolingAndCrossAt               'input_4[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

In [ ]:
history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=4, batch_size=32)

Epoch 1/4
84/84 [==============================] - 3857s 46s/step - loss: 1.0620 - categorical_accuracy: 0.5928 - val_loss: 0.5772 - val_categorical_accuracy: 0.7852
Epoch 2/4
84/84 [==============================] - 3867s 46s/step - loss: 0.4427 - categorical_accuracy: 0.8430 - val_loss: 0.3934 - val_categorical_accuracy: 0.8456
Epoch 3/4
47/84 [===============>..............] - ETA: 27:35 - loss: 0.2797 - categorical_accuracy: 0.9122

In [ ]:
result_bert = model.predict([test_input_ids,test_attention_masks])

In [ ]:
y_pred_bert =  np.zeros_like(result_bert)
y_pred_bert[np.arange(len(y_pred_bert)), result_bert.argmax(1)] = 1

In [ ]:
conf_matrix(y_test.argmax(1), y_pred_bert.argmax(1),'BERT Sentiment Analysis\nConfusion Matrix')

In [ ]:
print('\tClassification Report for BERT:\n\n',classification_report(y_test,y_pred_bert, target_names=['Negative', 'Neutral', 'Positive']))

ROBERTA MODEL

In [ ]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")

In [ ]:
token_lens = []

for txt in X_train:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
max_length

In [ ]:
MAX_LEN=128

In [ ]:
def tokenize_roberta(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
train_input_ids, train_attention_masks = tokenize_roberta(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize_roberta(X_valid, MAX_LEN)
test_input_ids, test_attention_masks = tokenize_roberta(X_test, MAX_LEN)

In [ ]:
def create_model(bert_model, max_len=MAX_LEN):
    
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()

    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(3, activation=tf.nn.softmax)(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(opt, loss=loss, metrics=accuracy)
    return model

In [ ]:
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

In [ ]:
model = create_model(roberta_model, MAX_LEN)
model.summary()

In [ ]:
history_2 = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=4, batch_size=30)

In [ ]:
result_roberta = model.predict([test_input_ids,test_attention_masks])

In [ ]:
y_pred_roberta =  np.zeros_like(result_roberta)
y_pred_roberta[np.arange(len(y_pred_roberta)), result_roberta.argmax(1)] = 1

In [ ]:
conf_matrix(y_test.argmax(1),y_pred_roberta.argmax(1),'RoBERTa Sentiment Analysis\nConfusion Matrix')

In [ ]:
print('\tClassification Report for RoBERTa:\n\n',classification_report(y_test,y_pred_roberta, target_names=['Negative', 'Neutral', 'Positive']))

In [ ]:

print('Classification Report for BERT:\n',classification_report(y_test,y_pred_bert, target_names=['Negative', 'Neutral', 'Positive']))

In [ ]:

print('Classification Report for RoBERTa:\n',classification_report(y_test,y_pred_roberta, target_names=['Negative', 'Neutral', 'Positive']))

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(9,5.5))

labels = ['Negative', 'Neutral', 'Positive']
plt.suptitle('Sentiment Analysis Comparison\n Confusion Matrix', fontsize=20)


sns.heatmap(confusion_matrix(y_test.argmax(1),y_pred_bert.argmax(1)), annot=True, cmap="Blues", fmt='g', cbar=False, ax=ax[0], annot_kws={"size":25})

ax[0].set_title('BERT Classifier', fontsize=20)
ax[0].set_yticklabels(labels, fontsize=17);
ax[0].set_xticklabels(labels, fontsize=17);
ax[0].set_ylabel('Test', fontsize=20)
ax[0].set_xlabel('Predicted', fontsize=20)

sns.heatmap(confusion_matrix(y_test.argmax(1),y_pred_roberta.argmax(1)), annot=True, cmap="Blues", fmt='g', cbar=False, ax=ax[1], annot_kws={"size":25})
ax[1].set_title('RoBERTa Classifier', fontsize=20)
ax[1].set_yticklabels(labels, fontsize=17);
ax[1].set_xticklabels(labels, fontsize=17);
ax[1].set_ylabel('Test', fontsize=20)
ax[1].set_xlabel('Predicted', fontsize=20)


plt.show()